In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
sc


<SparkContext master=local[*] appName=PySparkShell>

In [5]:
sc.stop()

In [6]:
spark.stop()

In [7]:
from pyspark import SparkConf, SparkContext
#setMaster() = Set Spark Content Manager which is local[cpu cores]
config = SparkConf().setMaster('local[4]').setAppName("ETL Pipeline")
sc = SparkContext(conf = config)

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ETL Pipeline').getOrCreate()

In [9]:
spark

In [10]:
sc

<SparkContext master=local[4] appName=ETL Pipeline>

In [11]:
hremployeeDF = spark.read.format("jdbc")\
.option("url", "jdbc:mysql://localhost:3306/hremployeeDB")\
.option("dbtable", "HR_Employee").option("user","root").option("password", "hadoop@123")\
.option("driver", "com.mysql.cj.jdbc.Driver").load()

In [12]:
hremployeeDF.show(5)

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|Attrition|Gender|Age|MaritalStatus|    Education|EducationField|   BusinessTravel|JobInvolvement|JobLevel|JobSatisfaction|Hourlyrate|Income|Salaryhike|OverTime|Workex|YearsSinceLastPromotion|EmpSatisfaction|TrainingTimesLastYear|WorkLifeBalance|Performance_Rating|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|
|
|
|
|
+----------+--------------------+

In [13]:
#Show physical plan of execution which is known as DAG.
#How data is scanned from the database
hremployeeDF.explain()

== Physical Plan ==
*(1) Scan JDBCRelation(HR_Employee) [numPartitions=1] [EmployeeID#0,Department#1,JobRole#2,Attrition#3,Gender#4,Age#5,MaritalStatus#6,Education#7,EducationField#8,BusinessTravel#9,JobInvolvement#10,JobLevel#11,JobSatisfaction#12,Hourlyrate#13,Income#14,Salaryhike#15,OverTime#16,Workex#17,YearsSinceLastPromotion#18,EmpSatisfaction#19,TrainingTimesLastYear#20,WorkLifeBalance#21,Performance_Rating#22] PushedFilters: [], ReadSchema: struct<EmployeeID:int,Department:string,JobRole:string,Attrition:string,Gender:string,Age:int,Mar...


#### Materialized view of Table

In [14]:
hremployeeDF.createOrReplaceTempView("hremployee")

#### 1.Display shape of hremployee tables
    * Show number of rows and number of columns

In [29]:
num_of_cols = len(hremployeeDF.columns)

In [30]:
spark.sql(f"""
SELECT COUNT(*) AS Rows, {num_of_cols} AS Columns FROM hremployee
""").show()

+----+-------+
|Rows|Columns|
+----+-------+
|1469|     23|
+----+-------+



spark.sql("""select count(*) from information_schema.columns where table_name = 'HR_Employee'

""").show()

In [33]:
spark.sql('describe hremployee').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          EmployeeID|      int|   null|
|          Department|   string|   null|
|             JobRole|   string|   null|
|           Attrition|   string|   null|
|              Gender|   string|   null|
|                 Age|      int|   null|
|       MaritalStatus|   string|   null|
|           Education|   string|   null|
|      EducationField|   string|   null|
|      BusinessTravel|   string|   null|
|      JobInvolvement|   string|   null|
|            JobLevel|      int|   null|
|     JobSatisfaction|   string|   null|
|          Hourlyrate|      int|   null|
|              Income|      int|   null|
|          Salaryhike|      int|   null|
|            OverTime|   string|   null|
|              Workex|      int|   null|
|YearsSinceLastPro...|      int|   null|
|     EmpSatisfaction|   string|   null|
+--------------------+---------+-------+
only showing top

#### 2. Write a query to show the first three employee from each Job role to join the company

In [44]:
spark.sql("""
select EmployeeID, JobRole, rank from
(select EmployeeID, JobRole, row_number() over(partition by JobRole order by EmployeeID asc) as rank
from hremployee)  as HRdata where rank <= 3
""").show()

+----------+--------------------+----+
|EmployeeID|             JobRole|rank|
+----------+--------------------+----+
|         1|     Sales Executive|   1|
|        28|     Sales Executive|   2|
|        40|     Sales Executive|   3|
|         9|Manufacturing Dir...|   1|
|        16|Manufacturing Dir...|   2|
|        21|Manufacturing Dir...|   3|
|         3|Laboratory Techni...|   1|
|         5|Laboratory Techni...|   2|
|         6|Laboratory Techni...|   3|
|        22|Sales Representative|   1|
|        34|Sales Representative|   2|
|        37|Sales Representative|   3|
|        10|Healthcare Repres...|   1|
|        29|Healthcare Repres...|   2|
|        32|Healthcare Repres...|   3|
|         2|  Research Scientist|   1|
|         4|  Research Scientist|   2|
|        13|  Research Scientist|   3|
|        19|             Manager|   1|
|        26|             Manager|   2|
+----------+--------------------+----+
only showing top 20 rows



#### 3. Write a query to show top 3 employee from each job role earning higher salary

In [46]:
spark.sql("""
select EmployeeID, JobRole,Income, rank from
(select EmployeeID, JobRole, Income, rank() over(partition by JobRole order by Income desc) as rank
from hremployee)  as HRdata where rank <= 3
""").show()

+----------+--------------------+------+----+
|EmployeeID|             JobRole|Income|rank|
+----------+--------------------+------+----+
|        99|     Sales Executive| 13872|   1|
|       545|     Sales Executive| 13770|   2|
|       839|     Sales Executive| 13758|   3|
|       722|Manufacturing Dir...| 13973|   1|
|       628|Manufacturing Dir...| 13826|   2|
|       744|Manufacturing Dir...| 13726|   3|
|       678|Laboratory Techni...|  7403|   1|
|       817|Laboratory Techni...|  6782|   2|
|       945|Laboratory Techni...|  6674|   3|
|       565|Sales Representative|  6632|   1|
|      1308|Sales Representative|  5405|   2|
|      1220|Sales Representative|  4502|   3|
|      1181|Healthcare Repres...| 13966|   1|
|       317|Healthcare Repres...| 13964|   2|
|       190|Healthcare Repres...| 13734|   3|
|        68|  Research Scientist|  9724|   1|
|      1315|  Research Scientist|  6962|   2|
|      1305|  Research Scientist|  6854|   3|
|       191|             Manager| 

#### 4. Show top 3 highest packages from overall job role

In [49]:
spark.sql("""
select jobrole, income from hremployee order by income desc limit 3 
""").show()

+-----------------+------+
|          jobrole|income|
+-----------------+------+
|          Manager| 19999|
|Research Director| 19973|
|          Manager| 19943|
+-----------------+------+



#### 5. Write a spark sql query to show employee in order of Ascending order with respect to employee income compared to previous income for each job role.

In [65]:
spark.sql("""
select EmployeeID, JobRole,income, LAG(income,1) over(order by JobRole) as prev,
income - LAG(income,1) over(order by JobRole) as diff
from hremployee 
order by jobrole, diff asc
""").show()

+----------+--------------------+------+-----+-----+
|EmployeeID|             JobRole|income| prev| diff|
+----------+--------------------+------+-----+-----+
|        10|Healthcare Repres...|  5237| null| null|
|       285|Healthcare Repres...|  4741|13496|-8755|
|      1183|Healthcare Repres...|  6842|13966|-7124|
|      1157|Healthcare Repres...|  4148|11245|-7097|
|       205|Healthcare Repres...|  6673|13734|-7061|
|       677|Healthcare Repres...|  4014|10552|-6538|
|       397|Healthcare Repres...|  4522|10965|-6443|
|       833|Healthcare Repres...|  5731|12169|-6438|
|      1065|Healthcare Repres...|  4035|10466|-6431|
|       745|Healthcare Repres...|  4777|10999|-6222|
|       736|Healthcare Repres...|  4240|10388|-6148|
|      1098|Healthcare Repres...|  4069|10124|-6055|
|        89|Healthcare Repres...|  4152|10096|-5944|
|       489|Healthcare Repres...|  4089| 9824|-5735|
|       929|Healthcare Repres...|  7978|13577|-5599|
|       105|Healthcare Repres...|  5163|10673|

#### lead()

In [68]:
spark.sql("""
select employeeid,department, jobrole, age, gender, income, workex,
lead(income, 2, 0) over(partition by jobrole order by employeeid) as next_income
from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|next_income|
+----------+----------+---------------+---+------+------+------+-----------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|       5376|
|        28|     Sales|Sales Executive| 42|  Male|  6825|    10|       8726|
|        40|     Sales|Sales Executive| 33|Female|  5376|    10|       4568|
|        44|     Sales|Sales Executive| 27|  Male|  8726|     9|       5772|
|        47|     Sales|Sales Executive| 34|  Male|  4568|    10|       5454|
|        49|     Sales|Sales Executive| 46|  Male|  5772|    14|       4157|
|        53|     Sales|Sales Executive| 44|Female|  5454|     9|       9069|
|        55|     Sales|Sales Executive| 26|Female|  4157|     5|       7637|
|        57|     Sales|Sales Executive| 35|  Male|  9069|     9|       5473|
|        64|     Sales|Sales Executive| 59|Female|  7637|    28|       4312|

#### ntile()
     * Dividing records into percentiles.

In [69]:
spark.sql("""
select employeeid,department, jobrole, age, gender, income, workex,
NTILE(4) over(order by income) as salary_quartiles
from hremployee
""").show()

+----------+--------------------+--------------------+---+------+------+------+----------------+
|employeeid|          department|             jobrole|age|gender|income|workex|salary_quartiles|
+----------+--------------------+--------------------+---+------+------+------+----------------+
|       514|Research & Develo...|  Research Scientist| 20|  Male|  1009|     1|               1|
|       728|Research & Develo...|  Research Scientist| 18|  Male|  1051|     0|               1|
|       765|               Sales|Sales Representative| 28|  Male|  1052|     1|               1|
|      1338|               Sales|Sales Representative| 30|  Male|  1081|     1|               1|
|      1365|               Sales|Sales Representative| 29|  Male|  1091|     1|               1|
|       178|Research & Develo...|Laboratory Techni...| 19|  Male|  1102|     1|               1|
|       912|               Sales|Sales Representative| 25|  Male|  1118|     1|               1|
|      1402|Research & Develo.

#### 6. Find the number of EMployee in each percentile group, 0-25th, 25th-50th, 50th-75th,75th-100th using percent_rank and case when .

In [78]:
spark.sql("""
with ranktable as
(select employeeid,
percent_rank() over(partition by department order by income)
as rank from hremployee) 

select rank_cat, count(rank_cat) from
(select employeeid, rank, 
case
    when rank < 0.25 then "0-25th"
    when rank < 0.50 then "25th-50th"
    when rank < 0.75 then "50th-75th"
    else "75th-100th"
end as rank_cat 
from ranktable)
as rankgrp 
group by rank_cat 
order by rank_cat
""").show()

+----------+---------------+
|  rank_cat|count(rank_cat)|
+----------+---------------+
|    0-25th|            367|
| 25th-50th|            366|
| 50th-75th|            367|
|75th-100th|            369|
+----------+---------------+



#### Hive Integration with Pyspark

In [79]:
spark.stop()

In [1]:
!jps

3489 SecondaryNameNode
2581 NodeManager
2389 ResourceManager
4645 Jps
2885 NameNode
3206 DataNode
4570 SparkSubmit


In [2]:
#Spark integration with Hive warehouse
#config name for Hive integration property name 'spark.sql.warehouse.dir'
#value = '/user/hive/warehouse'
spark = (SparkSession.builder.appName('pyspark-Hive=Integration')
        .config('spark.sql.warehouse.dir','/user/hive/warehouse')
        .enableHiveSupport().getOrCreate())

In [3]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|    airlines|
|     default|
+------------+



In [4]:
spark.sql("""
create database if not exists airlines
""")

DataFrame[]

In [6]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|    airlines|
|     default|
+------------+



In [4]:
spark.sql("""
use airlines
""")

DataFrame[]

In [35]:
spark.sql("""
create table if not exists flights(DayofMonth int, DayOfWeek int,
Carrier varchar(10), OriginAirportID int, DestAirportID int, DepDelay int, ArrDelay int)
row format delimited
fields terminated by ','
lines terminated by '\n'
stored as TEXTFILE
TBLPROPERTIES('skip.header.line.count'='1')
""")

DataFrame[]

In [34]:
spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|airlines| airports|      false|
+--------+---------+-----------+



In [36]:
spark.sql("""load data local inpath '/home/hadoop/Downloads/flights1.csv'
overwrite into table flights""")

DataFrame[]

In [20]:
spark.sql("""
create table if not exists airports(airport_id int, city varchar(50), state varchar(50), name varchar(50))
row format delimited
fields terminated by ','
lines terminated by '\n'
stored as TEXTFILE
TBLPROPERTIES('skip.header.line.count'='1')
""")

DataFrame[]

In [27]:
spark.sql("""load data local inpath '/home/hadoop/Downloads/airports_1.csv'
overwrite into table airports""")

DataFrame[]

In [38]:
spark.sql("select * from flights").show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [40]:
flights_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

#### 1.Extract

In [39]:
flights_df = spark.table('airlines.flights')
airports_df = spark.table('airlines.airports')

#### 2.Transformation

In [41]:
flights_join = flights_df.join(airports_df, on = flights_df.OriginAirportID == airports_df.airport_id, how = 'inner')

In [48]:
flights_join.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+----------+-----------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|       city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+-----------+-----+--------------------+
|        16|        1|     YV|          11057|        12264|      -6|     -19|     11057|  Charlotte|   NC|Charlotte Douglas...|
|        20|        4|     AS|          14747|        14679|      -9|     -12|     14747|    Seattle|   WA|Seattle/Tacoma In...|
|        12|        5|     UA|          11618|        15304|      48|      40|     11618|     Newark|   NJ|Newark Liberty In...|
|        14|        6|     EV|          14683|        11433|      -8|     -15|     14683|San Antonio|   TX|San Antonio Inter...|
|         4|        2|     AA|          14679|        11298|      -6|     -16|     14679|  San Di

#### 3.Load

In [43]:
flights_join=flights_join.repartition(4)

In [44]:
flights_join.write.parquet("file:///home/hadoop/Downloads/flights/")

In [45]:
#read a parquet file format
flights_parquet_df=spark.read.parquet("file:///home/hadoop/Downloads/flights/")

In [47]:
flights_join.write.partitionBy("Carrier").parquet("/airlines")

In [51]:
flights_join.write.partitionBy("Carrier").bucketBy(col = 'state', numBuckets = 30)\
.format("parquet").saveAsTable("part_bucket_table")

In [52]:
spark.sql("select Carrier, count(*) from part_bucket_table group by Carrier").show()

+-------+--------+
|Carrier|count(1)|
+-------+--------+
|     UA|  287601|
|     AA|  291771|
|     EV|  158253|
|     B6|  122297|
|     DL|  385040|
|     OO|  161102|
|     F9|   35821|
|     YV|   53022|
|     US|  235031|
|     MQ|  113634|
|     HA|   18658|
|     AS|   69056|
|     FL|   93013|
|     VX|   34869|
|     WN|  580029|
|     9E|   80221|
+-------+--------+



#### Load on MySQL

In [53]:
connection_properties = {
    'user':'root',
    'password' : 'hadoop@123',
    'driver' : 'com.mysql.cj.jdbc.Driver'
}

flights_join.write.jdbc(url = "jdbc:mysql://localhost:3306/flights", table ="airlines", mode = 'overwrite',
                       properties=connection_properties)

In [54]:
spark.stop()

In [55]:
sc.stop()